<a href="https://colab.research.google.com/github/yeon524/TEAM_PROJECT/blob/main/%EB%AC%BC%EB%A5%98_%EC%84%BC%ED%84%B0%EC%A0%9C%EC%95%BD_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================================
# 0. 라이브러리 & 데이터 로딩
# ================================
import math
import pandas as pd
import numpy as np
import pulp

# CSV 읽기 (파일 이름은 네가 가진 그대로 사용)
customers = pd.read_csv("Customer.csv")            # X, Y, DEMAND
centers   = pd.read_csv("DistributionCenter.csv")  # X, Y, CAPACITY, COST
trucks_df = pd.read_csv("Truck.csv")               # CAPACITY (160)

N = len(customers)   # 고객 수
M = len(centers)     # 센터 후보 수

truck_capacity = int(trucks_df.loc[0, "CAPACITY"])  # 트럭 적재용량 (예: 160)
total_demand   = customers["DEMAND"].sum()

print(f"고객 수: {N}, 센터 후보 수: {M}, 총 수요: {total_demand}, 트럭 용량: {truck_capacity}")

# ================================
# 1. 거리(DC–고객) 계산 (유클리드 거리)
# ================================
dist = np.zeros((M, N))  # dist[j, i] = 센터 j에서 고객 i까지 거리

for j in range(M):
    for i in range(N):
        dx = centers.loc[j, "X"] - customers.loc[i, "X"]
        dy = centers.loc[j, "Y"] - customers.loc[i, "Y"]
        dist[j, i] = math.hypot(dx, dy)  # sqrt(dx^2 + dy^2)

# ================================
# 2. 최적화 모델 정의
# ================================
model = pulp.LpProblem("FacilityLocation_With_Truck_Constraints", pulp.LpMinimize)

# 의사결정변수
# x[i,j] = 고객 i를 센터 j가 담당하면 1, 아니면 0
x = pulp.LpVariable.dicts(
    "assign",
    (range(N), range(M)),
    lowBound=0,
    upBound=1,
    cat="Binary",
)

# open[j] = 센터 j를 개설하면 1, 아니면 0
open_dc = pulp.LpVariable.dicts(
    "open",
    range(M),
    lowBound=0,
    upBound=1,
    cat="Binary",
)

# trucks[j] = 센터 j에서 출발하는 트럭 수 (정수)
trucks = pulp.LpVariable.dicts(
    "trucks",
    range(M),
    lowBound=0,
    cat="Integer",
)

# ================================
# 3. 목적함수: 센터 개설비용 + 총 이동거리
# ================================
fixed_cost = pulp.lpSum(
    open_dc[j] * centers.loc[j, "COST"] for j in range(M)
)

transport_cost = pulp.lpSum(
    dist[j, i] * x[i][j] for i in range(N) for j in range(M)
)

model += fixed_cost + transport_cost, "Total_Cost"

# ================================
# 4. 제약식
# ================================

# (1) 고객 배정 제약: 각 고객은 정확히 한 센터에 배정
for i in range(N):
    model += pulp.lpSum(x[i][j] for j in range(M)) == 1, f"assign_once_{i}"

# (2) 센터 CAPACITY 제약 (필요하면 유지, 지금은 크게 잡혀 있어서 거의 항상 여유)
for j in range(M):
    model += (
        pulp.lpSum(customers.loc[i, "DEMAND"] * x[i][j] for i in range(N))
        <= centers.loc[j, "CAPACITY"] * open_dc[j]
    ), f"center_capacity_{j}"

# (3) 센터 물량 편중 방지: 어떤 센터도 전체 수요의 50% 이상 담당 불가
for j in range(M):
    model += (
        pulp.lpSum(customers.loc[i, "DEMAND"] * x[i][j] for i in range(N))
        <= 0.5 * total_demand
    ), f"center_share_limit_{j}"

# (4) 센터 개설 효율 제약: 센터를 열면 최소 5명의 고객을 담당해야 함
MIN_CUSTOMERS_PER_CENTER = 5
for j in range(M):
    model += (
        pulp.lpSum(x[i][j] for i in range(N))
        >= MIN_CUSTOMERS_PER_CENTER * open_dc[j]
    ), f"center_min_customers_{j}"

# (5) 트럭 용량 제약: 센터 j에 할당된 수요 ≤ 트럭 수 * 트럭 용량
for j in range(M):
    model += (
        pulp.lpSum(customers.loc[i, "DEMAND"] * x[i][j] for i in range(N))
        <= trucks[j] * truck_capacity
    ), f"truck_capacity_{j}"

# (6) 센터가 안 열리면 트럭도 0 (Big-M)
BIG_M_TRUCKS = 100  # 충분히 큰 수 (센터당 최대 트럭 수 상한 비슷하게)
for j in range(M):
    model += trucks[j] <= BIG_M_TRUCKS * open_dc[j], f"truck_open_link_{j}"

# (7) 트럭 최대 이동거리 제약 (근사):
#     센터 j에서 담당하는 고객까지의 거리 합 ≤ 트럭 수 * 1대당 최대 이동거리
MAX_ROUTE_DIST_PER_TRUCK = 200  # 이 값은 시나리오에 맞게 조정 가능

for j in range(M):
    model += (
        pulp.lpSum(dist[j, i] * x[i][j] for i in range(N))
        <= trucks[j] * MAX_ROUTE_DIST_PER_TRUCK
    ), f"truck_max_distance_{j}"

# ================================
# 5. 풉! (모형 풀기)
# ================================
print("모형 푸는 중...")
solver = pulp.PULP_CBC_CMD(msg=True)  # msg=False로 하면 로그 안 나옴
result_status = model.solve(solver)

print("Status:", pulp.LpStatus[result_status])
print("최소 총비용:", pulp.value(model.objective))

# ================================
# 6. 해석: 어떤 센터가 열렸고, 트럭은 몇 대, 고객 배정은 어떻게?
# ================================
open_centers = []
for j in range(M):
    if pulp.value(open_dc[j]) > 0.5:
        open_centers.append(j)

print("\n=== 개설된 센터 목록 ===")
for j in open_centers:
    assigned_customers = [i for i in range(N) if pulp.value(x[i][j]) > 0.5]
    demand_j = sum(customers.loc[i, "DEMAND"] for i in assigned_customers)
    num_cust_j = len(assigned_customers)
    trucks_j = pulp.value(trucks[j])
    share_j = demand_j / total_demand * 100

    print(f"- 센터 {j} (좌표=({centers.loc[j,'X']}, {centers.loc[j,'Y']}))")
    print(f"  · 담당 고객 수: {num_cust_j}")
    print(f"  · 담당 수요 합: {demand_j} ({share_j:.1f}% of total)")
    print(f"  · 투입 트럭 수: {trucks_j}")
    print()

# (원하면 고객별로 어느 센터에 배정됐는지도 출력)
print("=== 고객별 배정 센터 ===")
for i in range(N):
    for j in range(M):
        if pulp.value(x[i][j]) > 0.5:
            print(f"고객 {i} -> 센터 {j}")
            break


고객 수: 50, 센터 후보 수: 5, 총 수요: 777, 트럭 용량: 160
모형 푸는 중...
Status: Optimal
최소 총비용: 1082.6937276264596

=== 개설된 센터 목록 ===
- 센터 1 (좌표=(37, 39))
  · 담당 고객 수: 15
  · 담당 수요 합: 195 (25.1% of total)
  · 투입 트럭 수: 100.0

- 센터 2 (좌표=(23, 43))
  · 담당 고객 수: 22
  · 담당 수요 합: 387 (49.8% of total)
  · 투입 트럭 수: 100.0

- 센터 3 (좌표=(56, 38))
  · 담당 고객 수: 13
  · 담당 수요 합: 195 (25.1% of total)
  · 투입 트럭 수: 2.0

=== 고객별 배정 센터 ===
고객 0 -> 센터 1
고객 1 -> 센터 2
고객 2 -> 센터 2
고객 3 -> 센터 2
고객 4 -> 센터 2
고객 5 -> 센터 2
고객 6 -> 센터 2
고객 7 -> 센터 2
고객 8 -> 센터 2
고객 9 -> 센터 3
고객 10 -> 센터 1
고객 11 -> 센터 1
고객 12 -> 센터 2
고객 13 -> 센터 2
고객 14 -> 센터 1
고객 15 -> 센터 3
고객 16 -> 센터 1
고객 17 -> 센터 2
고객 18 -> 센터 2
고객 19 -> 센터 3
고객 20 -> 센터 3
고객 21 -> 센터 1
고객 22 -> 센터 2
고객 23 -> 센터 2
고객 24 -> 센터 2
고객 25 -> 센터 2
고객 26 -> 센터 2
고객 27 -> 센터 1
고객 28 -> 센터 3
고객 29 -> 센터 3
고객 30 -> 센터 2
고객 31 -> 센터 1
고객 32 -> 센터 3
고객 33 -> 센터 3
고객 34 -> 센터 3
고객 35 -> 센터 3
고객 36 -> 센터 1
고객 37 -> 센터 1
고객 38 -> 센터 3
고객 39 -> 센터 2
고객 40 -> 센터 2
고객 41 -> 센터 1
고객 42 -> 센터 2
고객

In [5]:
import math
import pandas as pd
import numpy as np

customers = pd.read_csv("Customer.csv")            # X, Y, DEMAND
centers   = pd.read_csv("DistributionCenter.csv")  # X, Y, CAPACITY, COST

N = len(customers)
M = len(centers)
total_demand = customers["DEMAND"].sum()

print(f"고객 수: {N}, 센터 후보 수: {M}, 총 수요: {total_demand}")


고객 수: 50, 센터 후보 수: 5, 총 수요: 777


In [6]:
def build_distance_matrix(distance_type="euclidean"):
    dist = np.zeros((M, N))   # dist[j, i] = 센터 j -> 고객 i
    for j in range(M):
        for i in range(N):
            dx = centers.loc[j, "X"] - customers.loc[i, "X"]
            dy = centers.loc[j, "Y"] - customers.loc[i, "Y"]

            if distance_type == "euclidean":
                dist[j, i] = math.hypot(dx, dy)   # sqrt(dx^2 + dy^2)
            elif distance_type == "manhattan":
                dist[j, i] = abs(dx) + abs(dy)
            else:
                raise ValueError("distance_type은 'euclidean' 또는 'manhattan'만 사용하세요.")
    return dist


In [7]:
def compute_transport_cost(distance_type="euclidean"):
    dist = build_distance_matrix(distance_type)

    assignment = []   # 고객 i -> 센터 j
    total_cost = 0.0

    for i in range(N):
        all_dists = dist[:, i]                # 센터 j → 고객 i 거리 리스트
        best_j = int(np.argmin(all_dists))    # 가장 가까운 센터 index
        best_d = float(all_dists[best_j])     # 가장 가까운 거리

        demand_i = customers.loc[i, "DEMAND"]
        cost_i = best_d * demand_i            # 운송비 = 거리 × 수요

        assignment.append((i, best_j, best_d, demand_i, cost_i))
        total_cost += cost_i

    return total_cost, assignment


In [8]:
# 유클리드 기준 운송비
euclid_cost, euclid_assign = compute_transport_cost("euclidean")

# 맨해튼 기준 운송비
manhat_cost, manhat_assign = compute_transport_cost("manhattan")

print("=== 순수 운송비 비교 (제약 없음) ===")
print(f"EUCLIDEAN 기준 총 운송비 : {euclid_cost:.2f}")
print(f"MANHATTAN 기준 총 운송비 : {manhat_cost:.2f}")

print("\n[EUCLIDEAN 예시 5개]")
for i, j, d, dem, c in euclid_assign[:5]:
    print(f"고객 {i} -> 센터 {j}, 거리={d:.2f}, 수요={dem}, 운송비={c:.2f}")

print("\n[MANHATTAN 예시 5개]")
for i, j, d, dem, c in manhat_assign[:5]:
    print(f"고객 {i} -> 센터 {j}, 거리={d:.2f}, 수요={dem}, 운송비={c:.2f}")


=== 순수 운송비 비교 (제약 없음) ===
EUCLIDEAN 기준 총 운송비 : 13961.70
MANHATTAN 기준 총 운송비 : 17750.00

[EUCLIDEAN 예시 5개]
고객 0 -> 센터 0, 거리=9.22, 수요=7, 운송비=64.54
고객 1 -> 센터 2, 거리=15.23, 수요=30, 운송비=456.95
고객 2 -> 센터 2, 거리=29.70, 수요=16, 운송비=475.18
고객 3 -> 센터 2, 거리=28.60, 수요=9, 운송비=257.41
고객 4 -> 센터 2, 거리=26.42, 수요=21, 운송비=554.81

[MANHATTAN 예시 5개]
고객 0 -> 센터 0, 거리=13.00, 수요=7, 운송비=91.00
고객 1 -> 센터 2, 거리=20.00, 수요=30, 운송비=600.00
고객 2 -> 센터 2, 거리=42.00, 수요=16, 운송비=672.00
고객 3 -> 센터 2, 거리=40.00, 수요=9, 운송비=360.00
고객 4 -> 센터 2, 거리=36.00, 수요=21, 운송비=756.00


In [2]:
import math
import pandas as pd
import numpy as np
import pulp

# ================================
# 0. 데이터 로딩
# ================================
customers = pd.read_csv("Customer.csv")            # X, Y, DEMAND
centers   = pd.read_csv("DistributionCenter.csv")  # X, Y, CAPACITY, COST

N = len(customers)   # 고객 수
M = len(centers)     # 센터 후보 수
total_demand = customers["DEMAND"].sum()

print(f"고객 수: {N}, 센터 후보 수: {M}, 총 수요: {total_demand}")

# ================================
# 1. 거리 행렬 생성 함수
# ================================
def build_distance_matrix(distance_type="euclidean"):
    """
    distance_type: 'euclidean' 또는 'manhattan'
    반환: dist[j, i]  (센터 j -> 고객 i)
    """
    dist = np.zeros((M, N))
    for j in range(M):
        for i in range(N):
            dx = centers.loc[j, "X"] - customers.loc[i, "X"]
            dy = centers.loc[j, "Y"] - customers.loc[i, "Y"]

            if distance_type == "euclidean":
                dist[j, i] = math.hypot(dx, dy)  # sqrt(dx^2 + dy^2)
            elif distance_type == "manhattan":
                dist[j, i] = abs(dx) + abs(dy)
            else:
                raise ValueError("distance_type은 'euclidean' 또는 'manhattan'만 사용하세요.")
    return dist


고객 수: 50, 센터 후보 수: 5, 총 수요: 777


In [3]:
def solve_facility_model(distance_type="euclidean"):
    # 1) 거리 행렬 생성
    dist = build_distance_matrix(distance_type)

    # 2) 모형 정의
    model = pulp.LpProblem(
        f"FacilityLocation_{distance_type}",
        pulp.LpMinimize
    )

    # 의사결정변수
    # x[i,j] = 고객 i가 센터 j에 배정되면 1
    x = pulp.LpVariable.dicts(
        "assign",
        (range(N), range(M)),
        lowBound=0,
        upBound=1,
        cat="Binary",
    )

    # open[j] = 센터 j를 개설하면 1
    open_dc = pulp.LpVariable.dicts(
        "open",
        range(M),
        lowBound=0,
        upBound=1,
        cat="Binary",
    )

    # 3) 목적함수: 개설비용 + 운송비용
    fixed_cost = pulp.lpSum(
        open_dc[j] * centers.loc[j, "COST"] for j in range(M)
    )

    # 거리 × 수요 × 배정
    transport_cost = pulp.lpSum(
        dist[j, i] * customers.loc[i, "DEMAND"] * x[i][j]
        for i in range(N) for j in range(M)
    )

    model += fixed_cost + transport_cost, "TotalCost"

    # 4) 제약식

    # (1) 고객 배정: 각 고객은 정확히 하나의 센터에만 배정
    for i in range(N):
        model += pulp.lpSum(x[i][j] for j in range(M)) == 1, f"assign_once_{i}"

    # (2) 센터가 안 열리면 배정 불가
    for j in range(M):
        for i in range(N):
            model += x[i][j] <= open_dc[j], f"open_link_{i}_{j}"

    # (3) 센터 CAPACITY
    for j in range(M):
        model += (
            pulp.lpSum(customers.loc[i, "DEMAND"] * x[i][j] for i in range(N))
            <= centers.loc[j, "CAPACITY"] * open_dc[j]
        ), f"capacity_{j}"

    # (4) 센터 물량 편중 방지: 어떤 센터도 전체 수요의 50% 이상 담당 불가
    for j in range(M):
        model += (
            pulp.lpSum(customers.loc[i, "DEMAND"] * x[i][j] for i in range(N))
            <= 0.5 * total_demand
        ), f"share_limit_{j}"

    # (5) 센터 개설 효율: 열리면 최소 5명의 고객 담당
    MIN_CUSTOMERS = 5
    for j in range(M):
        model += (
            pulp.lpSum(x[i][j] for i in range(N))
            >= MIN_CUSTOMERS * open_dc[j]
        ), f"min_customers_{j}"

    # 5) 풀기
    print(f"\n===== {distance_type.upper()} 거리 기준 모형 풀이 시작 =====")
    solver = pulp.PULP_CBC_CMD(msg=False)
    status = model.solve(solver)
    print("Status:", pulp.LpStatus[status])

    if pulp.LpStatus[status] != "Optimal":
        print("최적해를 찾지 못했습니다.")
        return None

    total_cost = pulp.value(model.objective)
    # 개설비와 운송비 분리 계산
    fixed_val = sum(
        pulp.value(open_dc[j]) * centers.loc[j, "COST"] for j in range(M)
    )
    transport_val = sum(
        dist[j, i] * customers.loc[i, "DEMAND"] * pulp.value(x[i][j])
        for i in range(N) for j in range(M)
    )

    # 개설된 센터 요약
    centers_info = []
    for j in range(M):
        if pulp.value(open_dc[j]) > 0.5:
            assigned = [i for i in range(N) if pulp.value(x[i][j]) > 0.5]
            demand_j = sum(customers.loc[i, "DEMAND"] for i in assigned)
            share = demand_j / total_demand * 100
            centers_info.append({
                "id": j,
                "coord": (centers.loc[j, "X"], centers.loc[j, "Y"]),
                "num_customers": len(assigned),
                "demand": demand_j,
                "share_percent": share,
            })

    result = {
        "distance_type": distance_type,
        "total_cost": total_cost,
        "fixed_cost": fixed_val,
        "transport_cost": transport_val,
        "open_centers": centers_info,
    }
    return result


In [4]:
res_euclid = solve_facility_model("euclidean")
res_manhat = solve_facility_model("manhattan")

print("\n================ 비교 결과 ================")
for res in [res_euclid, res_manhat]:
    print(f"\n▶ 거리 기준: {res['distance_type'].upper()}")
    print(f"  - 총 비용       : {res['total_cost']:.2f}")
    print(f"    · 개설비용   : {res['fixed_cost']:.2f}")
    print(f"    · 운송비용   : {res['transport_cost']:.2f}")
    print(f"  - 개설된 센터 수: {len(res['open_centers'])}")
    for c in res["open_centers"]:
        print(f"    · 센터 {c['id']} @ {c['coord']}"
              f"  | 고객수={c['num_customers']},"
              f" 수요={c['demand']},"
              f" 비중={c['share_percent']:.1f}%")



===== EUCLIDEAN 거리 기준 모형 풀이 시작 =====
Status: Optimal

===== MANHATTAN 거리 기준 모형 풀이 시작 =====
Status: Optimal

================ 비교 결과 ================

▶ 거리 기준: EUCLIDEAN
  - 총 비용       : 14161.70
    · 개설비용   : 200.00
    · 운송비용   : 13961.70
  - 개설된 센터 수: 5
    · 센터 0 @ (np.int64(30), np.int64(46))  | 고객수=5, 수요=57, 비중=7.3%
    · 센터 1 @ (np.int64(37), np.int64(39))  | 고객수=7, 수요=82, 비중=10.6%
    · 센터 2 @ (np.int64(23), np.int64(43))  | 고객수=20, 수요=375, 비중=48.3%
    · 센터 3 @ (np.int64(56), np.int64(38))  | 고객수=13, 수요=195, 비중=25.1%
    · 센터 4 @ (np.int64(42), np.int64(43))  | 고객수=5, 수요=68, 비중=8.8%

▶ 거리 기준: MANHATTAN
  - 총 비용       : 17950.00
    · 개설비용   : 200.00
    · 운송비용   : 17750.00
  - 개설된 센터 수: 5
    · 센터 0 @ (np.int64(30), np.int64(46))  | 고객수=5, 수요=57, 비중=7.3%
    · 센터 1 @ (np.int64(37), np.int64(39))  | 고객수=6, 수요=79, 비중=10.2%
    · 센터 2 @ (np.int64(23), np.int64(43))  | 고객수=21, 수요=378, 비중=48.6%
    · 센터 3 @ (np.int64(56), np.int64(38))  | 고객수=12, 수요=172, 비중=22.1%
    · 센터 4 @ (np.i

In [9]:
def solve_with_share_limit(share_limit=0.5, distance_type="euclidean"):
    dist = build_distance_matrix(distance_type)

    model = pulp.LpProblem(f"share_limit_{share_limit}", pulp.LpMinimize)

    x = pulp.LpVariable.dicts("assign", (range(N), range(M)), 0, 1, cat='Binary')
    open_dc = pulp.LpVariable.dicts("open", range(M), 0, 1, cat='Binary')

    # === 목적함수 ===
    fixed_cost = pulp.lpSum(open_dc[j] * centers.loc[j, "COST"] for j in range(M))
    transport_cost = pulp.lpSum(
        dist[j,i] * customers.loc[i,"DEMAND"] * x[i][j]
        for i in range(N) for j in range(M)
    )
    model += fixed_cost + transport_cost

    # === 제약 ===
    for i in range(N):
        model += pulp.lpSum(x[i][j] for j in range(M)) == 1

    for j in range(M):
        for i in range(N):
            model += x[i][j] <= open_dc[j]

    # capacity
    for j in range(M):
        model += pulp.lpSum(customers.loc[i,"DEMAND"] * x[i][j] for i in range(N)) \
                <= centers.loc[j,"CAPACITY"]

    # ★ 여기! share_limit을 변수로 바꿈
    for j in range(M):
        model += pulp.lpSum(customers.loc[i,"DEMAND"] * x[i][j] for i in range(N)) \
                <= share_limit * total_demand

    # 최소 고객 5명 제약
    for j in range(M):
        model += pulp.lpSum(x[i][j] for i in range(N)) >= 5 * open_dc[j]

    model.solve(pulp.PULP_CBC_CMD(msg=False))

    # 결과 정리
    total_cost = pulp.value(model.objective)
    center_loads = []
    for j in range(M):
        if pulp.value(open_dc[j]) > 0.5:
            assigned = [i for i in range(N) if pulp.value(x[i][j]) > 0.5]
            demand = sum(customers.loc[i,"DEMAND"] for i in assigned)
            center_loads.append((j, demand))

    return {
        "share_limit": share_limit,
        "total_cost": total_cost,
        "num_centers": len(center_loads),
        "loads": center_loads,
    }


In [10]:
results = []
for limit in [0.4, 0.5, 0.6]:
    res = solve_with_share_limit(share_limit=limit)
    results.append(res)

for r in results:
    print("=== share_limit =", r["share_limit"], "===")
    print("총 비용:", r["total_cost"])
    print("개설 센터 수:", r["num_centers"])
    print("센터별 수요:", r["loads"])
    print()


=== share_limit = 0.4 ===
총 비용: 14299.340459294497
개설 센터 수: 5
센터별 수요: [(0, np.int64(84)), (1, np.int64(120)), (2, np.int64(310)), (3, np.int64(195)), (4, np.int64(68))]

=== share_limit = 0.5 ===
총 비용: 14161.700539085214
개설 센터 수: 5
센터별 수요: [(0, np.int64(57)), (1, np.int64(82)), (2, np.int64(375)), (3, np.int64(195)), (4, np.int64(68))]

=== share_limit = 0.6 ===
총 비용: 14161.700539085214
개설 센터 수: 5
센터별 수요: [(0, np.int64(57)), (1, np.int64(82)), (2, np.int64(375)), (3, np.int64(195)), (4, np.int64(68))]



In [5]:
import pandas as pd
import numpy as np

customers = pd.read_csv("Customer.csv")
centers = pd.read_csv("DistributionCenter.csv")

N = len(customers)
M = len(centers)


In [6]:
import pandas as pd
import numpy as np

# customers: customer.csv
# assignments: 고객 → 센터 번호 (길이 N 벡터)
# centers: distributionCenter.csv

def compute_center_share(customers, assignments, M):
    # assignments[i] = 고객 i가 배정된 센터 번호
    total_demand = customers["DEMAND"].sum()

    center_loads = [0] * M

    for i, c in enumerate(assignments):
        center_loads[c] += customers.loc[i, "DEMAND"]

    center_ratios = [load / total_demand for load in center_loads]

    return center_loads, center_ratios


# 예시) assignments가 있다고 가정 (너가 모델에서 나온 결과)
# assignments = [...]

# 사용 예시
loads, ratios = compute_center_share(customers, assignments, M)

print("=== 센터별 수요량 및 비중 ===")
for j in range(M):
    print(f"센터 {j} : 수요={loads[j]}, 비중={ratios[j]*100:.2f}%")

print("\n최대 비중 센터 =", np.argmax(ratios))
print("최대 비중 =", max(ratios)*100, "%")


NameError: name 'assignments' is not defined

In [7]:
def solve_with_share_limit(share_limit=0.5, distance_type="euclidean"):
    dist = build_distance_matrix(distance_type)

    model = pulp.LpProblem(f"share_limit_{share_limit}", pulp.LpMinimize)

    x = pulp.LpVariable.dicts("assign", (range(N), range(M)), 0, 1, cat='Binary')
    open_dc = pulp.LpVariable.dicts("open", range(M), 0, 1, cat='Binary')

    # 목적함수: 개설비 + 운송비
    fixed_cost = pulp.lpSum(open_dc[j] * centers.loc[j, "COST"] for j in range(M))
    transport_cost = pulp.lpSum(
        dist[j,i] * customers.loc[i,"DEMAND"] * x[i][j]
        for i in range(N) for j in range(M)
    )
    model += fixed_cost + transport_cost

    # (1) 각 고객은 정확히 1개 센터에 배정
    for i in range(N):
        model += pulp.lpSum(x[i][j] for j in range(M)) == 1

    # (2) 안 연 센터에는 배정 불가
    for j in range(M):
        for i in range(N):
            model += x[i][j] <= open_dc[j]

    # (3) 수용 CAPACITY
    for j in range(M):
        model += pulp.lpSum(customers.loc[i,"DEMAND"] * x[i][j] for i in range(N)) \
                 <= centers.loc[j,"CAPACITY"] * open_dc[j]

    # (4) 최대 비율 share_limit 제약
    for j in range(M):
        model += pulp.lpSum(customers.loc[i,"DEMAND"] * x[i][j] for i in range(N)) \
                 <= share_limit * total_demand

    # (5) 센터 열리면 최소 고객 5명
    MIN_CUSTOMERS = 5
    for j in range(M):
        model += pulp.lpSum(x[i][j] for i in range(N)) >= MIN_CUSTOMERS * open_dc[j]

    model.solve(pulp.PULP_CBC_CMD(msg=False))

    total_cost = pulp.value(model.objective)

    # 실제 각 센터 수요(비중 계산용)
    center_loads = [0] * M
    for j in range(M):
        if pulp.value(open_dc[j]) > 0.5:
            for i in range(N):
                if pulp.value(x[i][j]) > 0.5:
                    center_loads[j] += customers.loc[i, "DEMAND"]

    return {
        "share_limit": share_limit,
        "total_cost": total_cost,
        "center_loads": center_loads,
    }


In [8]:
import numpy as np

# 1) baseline: 거의 무제약 상태 비용 (예: 0.8)
baseline_res = solve_with_share_limit(share_limit=0.8)
baseline_cost = baseline_res["total_cost"]
print("baseline_cost =", baseline_cost)

total_demand = customers["DEMAND"].sum()

# 2) 후보 퍼센트들 (원하면 범위/간격 조절 가능)
candidate_limits = np.arange(0.30, 0.71, 0.02)  # 0.30, 0.32, ..., 0.70

def auto_choose_share_limit(candidates, tol=0.01):  # tol=0.01 → 비용 1% 이내 허용
    results = []

    for limit in candidates:
        res = solve_with_share_limit(share_limit=limit)
        loads = res["center_loads"]
        max_share = max(loads) / total_demand  # 실제 최대 비중
        gap = (res["total_cost"] - baseline_cost) / baseline_cost  # 비용 증가율

        results.append({
            "share_limit": limit,
            "total_cost": res["total_cost"],
            "max_share": max_share,
            "gap": gap,
            "loads": loads,
        })

    print("\n=== share_limit 튜닝 결과 ===")
    for r in results:
        print(f"share_limit={r['share_limit']:.2f} | "
              f"총비용={r['total_cost']:.2f} | "
              f"비용증가율={r['gap']*100:5.2f}% | "
              f"실제최대비중={r['max_share']*100:5.2f}% | "
              f"센터별수요={r['loads']}")

    # 3) baseline 대비 tol(예: 1%) 이내인 것만 후보로
    good = [r for r in results if r["gap"] <= tol]

    if good:
        # 그 중에서 share_limit(상한)이 가장 작은 것 선택 = 제일 강한 제약
        best = min(good, key=lambda r: r["share_limit"])
    else:
        # 없으면 그냥 비용이 제일 작은 것 선택
        best = min(results, key=lambda r: r["total_cost"])

    return best, results

best_res, all_res = auto_choose_share_limit(candidate_limits, tol=0.01)

print("\n👉 최종 추천 최대 비율(share_limit) =", best_res["share_limit"])
print("   해당 시 총비용 =", best_res["total_cost"])
print("   해당 시 실제 최대 비중 =", best_res["max_share"]*100, "%")
print("   센터별 수요 =", best_res["loads"])


NameError: name 'build_distance_matrix' is not defined